In [1]:
import os, sys, re, string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from data_preparation import take_a_shower, clean_content_aravec

In [2]:
data = pd.read_csv('./data/data_v1.csv', encoding="utf-16")
data.sample(10)

,Username,Comment,is_off,off_word
19603,Amal El Yaquine,نعم للزوجة ولا للعشيقة,0,0
36477,Relaxation And a variety of other contents.,هادي ماشي البارودي قدف الجبها هدي,0,0
23690,rabab en ligne,عجبتنی دیك تجمعهما صور لاتجمع الی من فی الحلال...,1,"0, لحرام"
22271,Om Reda reda,khasna nchofok f blog jdid 🤣🤣🤣🤣,0,0
24364,barakat. tv,مواضيع تافهة متبقاش تجيبوهم لينا اشوف تفي,0,0
4500,Maryam L3atmani,هاويلي 😂😂😂😂,0,0
34044,Desert eagle,Allah y sterna,0,0
11831,Mareym Malaki,لله يسهل على جميع ويبعدو على هد طريق,0,0
24689,Fadoua Mhila,مزيان ليك,0,0
20973,Ilham Cazal,اويلي فيناهيا لفورما ديال الرقص ويلي على شوهه ...,0,0


In [3]:
data_off = data.loc[data['is_off'] == 1, ['Comment', 'is_off']]
data_not_off = data.loc[data['is_off'] == 0, ['Comment', 'is_off']]

In [4]:
data_not_off = data_not_off.sample(n=3701)

In [18]:
data2 = pd.read_excel('./data/data_4000_selected.xlsx', engine='openpyxl')

In [19]:
data2.sample(10)

,Id,Platform,Comment,Majority_Label,Agreement,NumOfJudgementUsed,Total_Judgement,Vulgar:V/HateSpeech:HS/None:-
431,432,Twitter,@User.IDX أغرب خطأ لغوي قابلته هو: الجزيرة ؟!!...,Offensive,100.000000,3,3,-
3127,3128,Twitter,@User.IDX 😂😂😂😂😂😂😂🤣🤣🤣🤪🤦🏼‍♀️انتو لسه واقفين هنا ...,Non-Offensive,66.666667,3,3,-
3132,3133,YouTube,انا من الاردن يلي من الاردت يعطيني لايك يلا يا...,Non-Offensive,100.000000,3,4,-
3852,3853,Twitter,@User.IDX ك س ام ايران على قطر على سوريا على ل...,Offensive,100.000000,3,3,V
1849,1850,Facebook,أين صحفي سكاي نيوز عربي إسحاق المختفي في سوريا...,Non-Offensive,100.000000,3,3,-
2981,2982,Facebook,هي هاكدا ماتحتاج حد احسسها .انها طبيعتها .لانه...,Non-Offensive,100.000000,3,3,-
3147,3148,YouTube,ليش ما تتكلمون عن العراق ابدآ؟معقولة العراق ما...,Non-Offensive,100.000000,3,3,-
2293,2294,Twitter,@User.IDX 😂 😂 كلكم اخص والعن من بعض يا الشريفة...,Offensive,100.000000,3,3,-
2598,2599,YouTube,بتمنى تعمل حلقة عن السيسي وهو يحكي عن موضوع اع...,Non-Offensive,100.000000,3,3,-
2253,2254,Twitter,@User.IDX الميت لازم ياخذ أوسكار أفضل ممثل كوم...,Non-Offensive,100.000000,3,3,-


In [22]:
len(data2)

4000

In [23]:
data2_off = data2.loc[data2['Majority_Label'] == 'Offensive', ['Comment', 'Majority_Label']]
data2_not_off = data2.loc[data2['Majority_Label'] == 'Non-Offensive', ['Comment', 'Majority_Label']]

In [24]:
print(len(data2_off))
print(len(data2_not_off))

675
3325


In [25]:
data2_off.columns = ['Comment', 'is_off']
data2_not_off.columns = ['Comment', 'is_off']

In [27]:
data2_off['is_off'] = 1
data2_not_off['is_off'] = 0

In [29]:
data2_not_off = data2_not_off.sample(n=675)

In [30]:
print(len(data2_off))
print(len(data2_not_off))

675
675


In [51]:
data_aj = pd.read_excel('./data/data_AJ_comments.xlsx', engine='openpyxl')

In [52]:
data_aj.sample(5)

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,languagecomment,languagecomment:confidence,articletitle,body,bodylen,insdt,languagecomment_gold,link,serial,words
18788,1222302629,False,finalized,3,4/30/2017 14:26:58,-1,1.0000,تسجيل لتعذيب النظام جنوده الفارين من مواجهة ال...,بدل هؤلاء الجنود يفتحون النار علىقادة حماة الص...,70,2015-06-01 02:19:08.573,NaN,www.aljazeera.net/home/getpage/f6451603-4dff-4...,49013,10
16838,1222309071,False,finalized,3,5/2/2017 14:36:34,-1,1.0000,مجندات بلا خبرة يواجهن ثوار سوريا,تركوا جر الذيول وخلوها للعرعور اللي يجر بشته و...,87,2015-04-20 04:59:12.483,NaN,www.aljazeera.net/home/getpage/f6451603-4dff-4...,62522,13
27828,1222282735,False,finalized,3,4/28/2017 11:41:20,-1,1.0000,قتيلان بهجوم على حسينية في إيران,أيران خوزستان المجوسيه‏!‏‏!‏ الدوله الاسلاميه ...,75,2015-10-17 11:34:23.343,NaN,www.aljazeera.net/home/getpage/f6451603-4dff-4...,8708,11
21165,1222294887,False,finalized,3,4/29/2017 12:06:57,-1,0.6851,نيويورك تايمز: تنظيم الدولة يؤسس لدولة ناجحة,و لا الساحل مقر بنو صهيون,25,2015-07-23 08:06:05.130,NaN,www.aljazeera.net/home/getpage/f6451603-4dff-4...,33695,6
12552,1222280614,False,finalized,3,4/28/2017 09:34:00,-1,1.0000,مطالب برفع يد الكنيسة عن مسجد قرطبة,ليفجروه بعملية انتحارية ..خلوه كما هو احسن .,44,2015-11-04 08:47:54.330,NaN,www.aljazeera.net/home/getpage/f6451603-4dff-4...,4638,8


In [53]:
len(data_aj)

31692

In [54]:
data_aj_off = data_aj.loc[data_aj['languagecomment'] != 0, ['body', 'languagecomment']]
data_aj_not_off = data_aj.loc[data_aj['languagecomment'] == 0, ['body', 'languagecomment']]

In [55]:
print(len(data_aj_off))
print(len(data_aj_not_off))

26039
5653


In [56]:
data_aj_off.columns = ['Comment', 'is_off']
data_aj_not_off.columns = ['Comment', 'is_off']

In [57]:
data_aj_off['is_off'] = 1
data_aj_not_off['is_off'] = 0

In [58]:
data_aj_off = data_aj_off.sample(n=3624)
data_aj_not_off = data_aj_not_off.sample(n=3624)

In [59]:
print("MA  : " + str(len(data_off)) + "\nEGY : " + str(len(data2_off)) + "\nGolf: " + str(len(data_aj_off)))
print("Total Offensive Data : % 2d" %(len(data_off) + len(data2_off) + len(data_aj_off)))

MA  : 3701
EGY : 675
Golf: 3624
Total Offensive Data :  8000


In [60]:
dataset_offensive = pd.concat([data_off, data2_off, data_aj_off])

In [61]:
dataset_offensive.shape

(8000, 2)

In [62]:
print("MA  : " + str(len(data_not_off)) + "\nGolf: " + str(len(data2_not_off)) + "\nGolf: " + str(len(data_aj_not_off)))
print("Total Non-Offensive Data : % 2d" %(len(data_not_off) + len(data2_not_off) + len(data_aj_not_off)))

MA  : 3701
Golf: 675
Golf: 3624
Total Non-Offensive Data :  8000


In [63]:
dataset_not_offensive_all = pd.concat([data_not_off, data2_not_off, data_aj_not_off])

In [72]:
dataset_not_offensive_all

,Comment,is_off
19266,Una kamlin mzowjin Samui lia,0
23822,مراتو زعمة مطورة ههههههه راه خاصك تربيه هو ماش...,0
15628,5oiya mdloma Bear .bhalk a5oia mkainch,0
738,كنحس بيها محروقة من الداخل ديالها اومتوثرة غي ...,0
12488,عقلو عليهم... 08.02.2019 غادي يدرو لبااس و يسر...,0
...,...,...
27836,ماهو العراده من الجوف.وهي محافظه كبيره وفيها م...,0
8975,هل يفيد المشمش في علاج أعراض الشيخوخة ، شكرا ...,0
1252,معلش يا ابن عمتي ارجو تصحيح اسمي في التعليق ال...,0
12487,كيف القتل لتكفير ملة إينما كانو والقتل كردة فع...,0


In [73]:
offensive_dataset = pd.concat([dataset_offensive, dataset_not_offensive_all]).sample(frac=1).reset_index(drop=True)

In [74]:
offensive_dataset

,Comment,is_off
0,برتوش ههههههه علام,1
1,معرفتش اعلاش حسيتها كتكدب,0
2,شوفوا يا برجوازيين البلاد ل كيهربوا لفلوس لدو...,0
3,ليش ما يعطوهم سفري .,0
4,ساقاطع هذا الموقع الذي هدفه نشر الكراهية تجاه ...,0
...,...,...
15995,هل قصدك بالأسهم Binary option trading و لا الم...,0
15996,لاداعي لرمي الاتهماة في تغصير الامني ويجب اتوى...,0
15997,اتضح من الريش المتطاير انو هادا كان سرب طيور ...,1
15998,اما تنظيم الدولة فامريكاوروسيا من صنعته لكى لا...,1


In [75]:
offensive_dataset.loc[ 13851 , : ]

Comment    كضحكو على الوقت بقاو لينا غير الشيخات ها الناش...
is_off                                                     1
Name: 13851, dtype: object

In [76]:
offensive_dataset.to_csv('./data/offensive_dataset.csv', index=False, encoding="utf-8")